# Simulação da Equação da Difusão

Este notebook apresenta uma simulação simplificada da equação da difusão, também conhecida como a segunda lei de Fick ou equação do calor.

## Teoria

A equação da difusão descreve como a concentração de uma substância muda ao longo do tempo devido ao movimento aleatório das partículas. Em sua forma mais simples, a equação é dada por:

$$\frac{\partial u}{\partial t} = D \nabla^2 u$$

Onde:
- $u(x,t)$ é a concentração da substância (ou temperatura, no caso da equação do calor)
- $t$ é o tempo
- $D$ é o coeficiente de difusão
- $\nabla^2$ é o operador Laplaciano

Em uma dimensão, a equação se torna:

$$\frac{\partial u}{\partial t} = D \frac{\partial^2 u}{\partial x^2}$$

Em duas dimensões:

$$\frac{\partial u}{\partial t} = D \left(\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}\right)$$

## Método Numérico

Para resolver numericamente a equação da difusão, utilizamos o método de diferenças finitas. Discretizamos o espaço e o tempo, e aproximamos as derivadas por diferenças finitas.

Para a derivada temporal, usamos a aproximação forward:

$$\frac{\partial u}{\partial t} \approx \frac{u(x,t+\Delta t) - u(x,t)}{\Delta t}$$

Para a derivada espacial de segunda ordem, usamos a aproximação central:

$$\frac{\partial^2 u}{\partial x^2} \approx \frac{u(x+\Delta x,t) - 2u(x,t) + u(x-\Delta x,t)}{(\Delta x)^2}$$

Substituindo na equação da difusão, obtemos o esquema numérico:

$$\frac{u(x,t+\Delta t) - u(x,t)}{\Delta t} = D \frac{u(x+\Delta x,t) - 2u(x,t) + u(x-\Delta x,t)}{(\Delta x)^2}$$

Isolando $u(x,t+\Delta t)$:

$$u(x,t+\Delta t) = u(x,t) + D\frac{\Delta t}{(\Delta x)^2}[u(x+\Delta x,t) - 2u(x,t) + u(x-\Delta x,t)]$$

Este é o esquema explícito de diferenças finitas para a equação da difusão.

In [ ]:
# Importando as bibliotecas necessárias
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Configurações do Seaborn para o estilo escuro
sns.set(style='darkgrid')

## Simulação em 1D

Vamos começar com uma simulação unidimensional da equação da difusão.

In [ ]:
# Parâmetros da simulação 1D
L = 10.0        # Comprimento do domínio
N = 100         # Número de pontos no espaço
dx = L / N      # Passo no espaço
D = 1.0         # Coeficiente de difusão
dt = 0.01       # Passo no tempo
T = 2.0         # Tempo total de simulação
n_steps = int(T / dt)  # Número de passos no tempo

# Verificação da estabilidade numérica
stability_factor = D * dt / dx**2
print(f"Fator de estabilidade: {stability_factor:.4f}")
if stability_factor > 0.5:
    print("AVISO: O esquema numérico pode ser instável! Recomenda-se reduzir dt ou aumentar dx.")
else:
    print("O esquema numérico é estável.")

# Criação do domínio espacial
x = np.linspace(0, L, N)

# Condições iniciais
def initial_condition_pulse(x, L):
    """Pulso concentrado no centro"""
    u = np.zeros_like(x)
    u[N//2] = 1.0 / dx  # Pico no centro
    return u

def initial_condition_gaussian(x, L, sigma=0.5):
    """Distribuição gaussiana centrada"""
    return np.exp(-((x - L/2)**2) / (2 * sigma**2)) / (sigma * np.sqrt(2 * np.pi))

def initial_condition_step(x, L):
    """Função degrau"""
    u = np.zeros_like(x)
    u[x < L/2] = 1.0
    return u

# Escolha da condição inicial
u = initial_condition_gaussian(x, L)

# Função para atualizar a distribuição no tempo (esquema explícito)
def update_1d(u, D, dx, dt):
    u_new = u.copy()
    # Aplicando o esquema de diferenças finitas para pontos internos
    for i in range(1, N-1):
        u_new[i] = u[i] + D * dt / dx**2 * (u[i+1] - 2*u[i] + u[i-1])
    return u_new

# Armazenando a evolução temporal para visualização
u_history = np.zeros((n_steps + 1, N))
u_history[0] = u.copy()

# Simulação
for step in range(n_steps):
    u = update_1d(u, D, dx, dt)
    u_history[step + 1] = u.copy()

# Visualização da evolução temporal em diferentes instantes
plt.figure(figsize=(10, 6))
time_points = [0, int(n_steps/4), int(n_steps/2), n_steps]
for i, t in enumerate(time_points):
    plt.plot(x, u_history[t], label=f't = {t*dt:.2f}')

plt.xlabel('Posição (x)')
plt.ylabel('Concentração (u)')
plt.title('Evolução da Difusão em 1D')
plt.legend()
plt.grid(True)
plt.show()

### Animação da Difusão em 1D

In [ ]:
# Configuração da figura para animação
fig, ax = plt.subplots(figsize=(10, 6))
line, = ax.plot(x, u_history[0], lw=2)
ax.set_ylim(0, max(u_history[0]) * 1.1)
ax.set_xlabel('Posição (x)')
ax.set_ylabel('Concentração (u)')
ax.set_title('Evolução da Difusão ao Longo do Tempo')

# Função de animação
def animate_1d(frame):
    line.set_ydata(u_history[frame])
    ax.set_title(f'Evolução da Difusão - t = {frame*dt:.2f}')
    return line,

# Criação da animação
ani = FuncAnimation(fig, animate_1d, frames=n_steps+1, interval=50, blit=True)

# Exibição da animação
HTML(ani.to_jshtml())

## Simulação em 2D

Agora, vamos estender nossa simulação para duas dimensões.

In [ ]:
# Parâmetros da simulação 2D
L_2d = 5.0       # Comprimento do domínio quadrado
N_2d = 50        # Número de pontos em cada direção
dx_2d = L_2d / N_2d  # Passo no espaço
D_2d = 1.0       # Coeficiente de difusão
dt_2d = 0.005    # Passo no tempo (reduzido para garantir estabilidade em 2D)
T_2d = 1.0       # Tempo total de simulação
n_steps_2d = int(T_2d / dt_2d)  # Número de passos no tempo

# Verificação da estabilidade numérica para 2D
stability_factor_2d = D_2d * dt_2d / dx_2d**2
print(f"Fator de estabilidade 2D: {stability_factor_2d:.4f}")
if stability_factor_2d > 0.25:  # Em 2D, o limite é mais restritivo
    print("AVISO: O esquema numérico pode ser instável em 2D! Recomenda-se reduzir dt ou aumentar dx.")
else:
    print("O esquema numérico é estável em 2D.")

# Criação do domínio espacial 2D
x_2d = np.linspace(0, L_2d, N_2d)
y_2d = np.linspace(0, L_2d, N_2d)
X, Y = np.meshgrid(x_2d, y_2d)

# Condição inicial 2D: fonte pontual no centro
u_2d = np.zeros((N_2d, N_2d))
center = N_2d // 2
sigma = 0.3 * L_2d
u_2d = np.exp(-((X - L_2d/2)**2 + (Y - L_2d/2)**2) / (2 * sigma**2))

# Função para atualizar a distribuição 2D no tempo (esquema explícito)
def update_2d(u, D, dx, dt):
    u_new = u.copy()
    # Aplicando o esquema de diferenças finitas para pontos internos
    for i in range(1, N_2d-1):
        for j in range(1, N_2d-1):
            laplacian = (u[i+1, j] + u[i-1, j] + u[i, j+1] + u[i, j-1] - 4*u[i, j]) / dx**2
            u_new[i, j] = u[i, j] + D * dt * laplacian
    return u_new

# Armazenando alguns frames para visualização
n_frames = 5
frame_indices = np.linspace(0, n_steps_2d, n_frames, dtype=int)
u_frames = np.zeros((n_frames, N_2d, N_2d))
u_frames[0] = u_2d.copy()

# Simulação 2D
u_current = u_2d.copy()
for step in range(1, n_steps_2d + 1):
    u_current = update_2d(u_current, D_2d, dx_2d, dt_2d)
    if step in frame_indices:
        idx = np.where(frame_indices == step)[0][0]
        u_frames[idx] = u_current.copy()

# Visualização de frames selecionados
fig, axes = plt.subplots(1, n_frames, figsize=(15, 4))
for i in range(n_frames):
    im = axes[i].imshow(u_frames[i], cmap='viridis', origin='lower', 
                        extent=[0, L_2d, 0, L_2d], vmin=0, vmax=u_frames[0].max())
    axes[i].set_title(f't = {frame_indices[i]*dt_2d:.2f}')
    axes[i].set_xlabel('x')
    if i == 0:
        axes[i].set_ylabel('y')

plt.tight_layout()
fig.colorbar(im, ax=axes.ravel().tolist(), label='Concentração')
plt.show()

### Animação da Difusão em 2D

In [ ]:
# Reiniciando a simulação para a animação
u_2d = np.exp(-((X - L_2d/2)**2 + (Y - L_2d/2)**2) / (2 * sigma**2))

# Configuração da figura para animação 2D
fig, ax = plt.subplots(figsize=(8, 8))
im = ax.imshow(u_2d, cmap='viridis', origin='lower', 
               extent=[0, L_2d, 0, L_2d], vmin=0, vmax=u_2d.max())
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Difusão 2D - t = 0.00')
fig.colorbar(im, ax=ax, label='Concentração')

# Número reduzido de frames para a animação (para economizar memória)
n_frames_anim = 100
steps_per_frame = max(1, n_steps_2d // n_frames_anim)

# Função de animação 2D
def animate_2d(frame):
    global u_2d
    for _ in range(steps_per_frame):
        u_2d = update_2d(u_2d, D_2d, dx_2d, dt_2d)
    im.set_array(u_2d)
    ax.set_title(f'Difusão 2D - t = {frame*steps_per_frame*dt_2d:.2f}')
    return [im]

# Criação da animação 2D
ani_2d = FuncAnimation(fig, animate_2d, frames=n_frames_anim, interval=100, blit=True)

# Exibição da animação 2D
HTML(ani_2d.to_jshtml())

## Comparação com Solução Analítica

Para alguns casos especiais, a equação da difusão possui soluções analíticas. Vamos comparar nossa solução numérica com a solução analítica para o caso de uma fonte pontual em um domínio infinito.

A solução analítica para uma fonte pontual em 1D é dada por:

$$u(x,t) = \frac{M}{\sqrt{4\pi Dt}} \exp\left(-\frac{(x-x_0)^2}{4Dt}\right)$$

Onde $M$ é a massa total e $x_0$ é a posição inicial da fonte.

In [ ]:
# Parâmetros para comparação
L_comp = 20.0     # Domínio maior para aproximar um domínio infinito
N_comp = 200      # Mais pontos para maior precisão
dx_comp = L_comp / N_comp
D_comp = 1.0
dt_comp = 0.005   # Passo de tempo menor para maior precisão
T_comp = 2.0
n_steps_comp = int(T_comp / dt_comp)

# Domínio espacial
x_comp = np.linspace(0, L_comp, N_comp)

# Condição inicial: pulso gaussiano estreito
sigma_init = 0.5
x0 = L_comp / 2  # Centro do domínio
u_comp = np.exp(-((x_comp - x0)**2) / (2 * sigma_init**2)) / (sigma_init * np.sqrt(2 * np.pi))
M = dx_comp * np.sum(u_comp)  # Massa total (integral da condição inicial)

# Função para a solução analítica
def analytical_solution(x, t, D, M, x0):
    if t == 0:
        # Evitar divisão por zero
        result = np.zeros_like(x)
        idx = np.argmin(np.abs(x - x0))
        result[idx] = M / dx_comp
        return result
    return M / np.sqrt(4 * np.pi * D * t) * np.exp(-((x - x0)**2) / (4 * D * t))

# Simulação numérica
u_num = u_comp.copy()
times_to_compare = [0, 0.1, 0.5, 1.0, 2.0]  # Tempos para comparação
step_indices = [min(n_steps_comp, int(t / dt_comp)) for t in times_to_compare]

# Armazenar resultados numéricos
u_num_results = {}
u_num_results[0] = u_num.copy()

# Executar simulação
for step in range(1, n_steps_comp + 1):
    u_num = update_1d(u_num, D_comp, dx_comp, dt_comp)
    if step in step_indices:
        t_idx = times_to_compare[step_indices.index(step)]
        u_num_results[t_idx] = u_num.copy()

# Comparação entre solução numérica e analítica
plt.figure(figsize=(12, 8))
for i, t in enumerate(times_to_compare):
    if t in u_num_results:
        plt.subplot(len(times_to_compare), 1, i+1)
        plt.plot(x_comp, u_num_results[t], 'b-', label='Numérica')
        u_ana = analytical_solution(x_comp, t, D_comp, M, x0)
        plt.plot(x_comp, u_ana, 'r--', label='Analítica')
        plt.title(f't = {t}')
        plt.xlabel('x')
        plt.ylabel('u')
        plt.legend()
        plt.grid(True)

plt.tight_layout()
plt.show()

## Conclusão

Neste notebook, apresentamos uma simulação simplificada da equação da difusão em uma e duas dimensões. Utilizamos o método de diferenças finitas para resolver numericamente a equação e visualizamos a evolução temporal da concentração.

Observamos que:

1. A difusão tende a espalhar a concentração ao longo do tempo, de regiões de alta concentração para regiões de baixa concentração.
2. A taxa de difusão é controlada pelo coeficiente de difusão $D$.
3. A solução numérica se aproxima da solução analítica para casos simples.
4. A estabilidade numérica é crucial para obter resultados precisos.

Esta simulação pode ser estendida para incluir condições de contorno mais complexas, difusão anisotrópica, termos de fonte/sumidouro, e muito mais.